# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [7]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama
# import pdfplumber
# import requests
# from io import BytesIO


In [16]:
# constants
OLLAMA_API = "http://localhost:11434/v1"
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [9]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI()


API key looks good so far


In [10]:
# here is the question; type over this to ask something new

system_prompt="""
You are a helpful technical support assistant.
"""

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
""" 


In [11]:
# Get gpt-4o-mini to answer, with streaming

def answer_openai(question, stream_response=False):
    response = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=stream_response
    )
    if stream_response:
        result = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in response:
            result += chunk.choices[0].delta.content or ''
            result = result.replace("```","").replace("markdown", "")
            update_display(Markdown(result), display_id=display_handle.display_id)
    else:
        result = response.choices[0].message.content
        display(Markdown(result))

answer_openai(question,stream_response=True)

The code you've provided is a Python expression that utilizes the `yield from` statement in conjunction with a set comprehension. Let's break it down step by step.

### Components of the Code

1. **Set Comprehension**:
   - `{book.get("author") for book in books if book.get("author")}`
   - This part iterates over a collection called `books`, which is expected to be an iterable (like a list or a tuple) containing dictionaries (or objects) representing books.
   - For each `book` in `books`, it attempts to retrieve the value associated with the key `"author"` using `book.get("author")`.
   - The `if book.get("author")` condition ensures that only entries where the `"author"` key exists and has a truthy value (not `None`, `''`, `0`, etc.) are included in the resulting set.
   - The result of this comprehension is a set of unique authors (as sets automatically handle duplicates).

2. **`yield from`**:
   - The `yield from` statement is used in a generator function to yield all values from an iterable. This means it will yield each element from the set created by the set comprehension one at a time.
   - Using `yield from` helps in maintaining the state of the generator and simplifies the code when delegating to sub-generators or iterables.

### What the Entire Code Does

Putting it all together, this code snippet defines a part of a generator function that:
- Extracts all unique authors from the `books` iterable.
- Only includes authors that are present and have a truthy value.
- Yields each unique author one by one whenever this generator function is called.

### Example Scenario

Suppose you have a list of books like this:

python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": None},
    {"title": "Book 4", "author": "Author A"},
    {"title": "Book 5", "author": ""},
]


In this case, the set comprehension will create a set with `{"Author A", "Author B"}`. Then, when the generator function containing `yield from` runs, it will yield "Author A" first, followed by "Author B".

### Summary

- The code extracts unique author names from a collection of book dictionaries, skipping any entries where the author is missing or falsy.
- It uses the generator pattern to yield each author sequentially, making it efficient in terms of memory and performance when dealing with potentially large datasets.

In [18]:
# Get Llama 3.2 to answer
# Use OPENAI API for consistency

def answer_ollama(question, stream_response=False):
    ollama_via_openai = OpenAI(base_url=OLLAMA_API, api_key='ollama')
    response = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=stream_response
    )
    if stream_response:
        result = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in response:
            result += chunk.choices[0].delta.content or ''
            result = result.replace("```","").replace("markdown", "")
            update_display(Markdown(result), display_id=display_handle.display_id)
    else:
        result = response.choices[0].message.content
        display(Markdown(result))

answer_ollama(question)

**Generative Code Explanation**

The given line of code is a generator expression used to iterate over a list of dictionaries (`books`) and extract the "author" field from each dictionary. Let's break it down:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

**Step-by-Step Explanation**

1. `{book.get("author") for book in books}`: This is a dictionary comprehension that iterates over the `books` list, fetching the value associated with the key `"author"` from each dictionary.
2. `if book.get("author")`: This adds an additional condition to filter out any dictionaries where `"author"` doesn't exist.
3. `yield from ...`: The `yield from` keyword is used in Python 3.3 and later versions. It allows a generator function to yield results from another generator or iterable.

**What it does**

This expression creates a new generator that yields the values of only those dictionaries (`book`) where `"author"` exists. However, this line cannot be part of a single line within Python code because `yield from` needs to be used as a top-level statement or inside a function for its full functionality.

**Corrected Usage**

If we wanted to achieve the same result in a more traditional generator expression:

```python
def books_authors(books):
    for book in books:
        if book.get("author"):
            yield book["author"]

# Example usage
books = [
    {"author": "J.K. Rowling", "title": "Harry Potter"},
    {"book_info": "Not an author-related field"},
    {"author": "John Green", "publisher": "Penguin Press"}
]

for author in books_authors(books):
    print(author)
```

In this corrected code:

- We define a generator function `books_authors` that yields the authors' names from dictionaries with existing `"author"` keys.
- We create an instance of the generator and use it to extract authors from the list.